In [1]:
!which python

/vol/ml/apartin/anaconda3/envs/p1/bin/python


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from glob import glob

import sklearn
import numpy as np
import pandas as pd
from math import sqrt

In [3]:
runs_dir_name = 'out_lgbm'
file_path = Path.cwd()
print(file_path)

/vol/ml/apartin/projects/DrugResponseViz/notebooks


In [4]:
runs_dir_path = Path(file_path / '..' / runs_dir_name)
runs_dirs = [Path(p) for p in glob(str(runs_dir_path/'run_*'))]

In [6]:
def calc_scores(y_true, y_pred, mltype, metrics=None):
    """ Create dict of scores.
    Args:
        metrics : TODO allow to pass a string of metrics
    """
    import sklearn.metrics as metrics
    scores = {}

    if mltype == 'cls':    
        scores['auroc'] = sklearn.metrics.roc_auc_score(y_true, y_pred)
        scores['f1_score'] = sklearn.metrics.f1_score(y_true, y_pred, average='micro')
        scores['acc_blnc'] = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)

    elif mltype == 'reg':
#         scores['r2'] = sklearn.metrics.r2_score(y_true=y_true, y_pred=y_pred)
#         scores['mean_absolute_error'] = sklearn.metrics.mean_absolute_error(y_true=y_true, y_pred=y_pred)
#         scores['median_absolute_error'] = sklearn.metrics.median_absolute_error(y_true=y_true, y_pred=y_pred)
#         # scores['mean_squared_error'] = sklearn.metrics.mean_squared_error(y_true=y_true, y_pred=y_pred)
#         scores['mse'] = sklearn.metrics.mean_squared_error(y_true=y_true, y_pred=y_pred)
#         scores['rmse'] = sqrt( sklearn.metrics.mean_squared_error(y_true=y_true, y_pred=y_pred) )
#         # scores['auroc_reg'] = reg_auroc(y_true=y_true, y_pred=y_pred)
        
        scores['r2'] = metrics.r2_score(y_true=y_true, y_pred=y_pred)
        scores['mean_absolute_error'] = metrics.mean_absolute_error(y_true=y_true, y_pred=y_pred)
        scores['median_absolute_error'] = metrics.median_absolute_error(y_true=y_true, y_pred=y_pred)
        # scores['mean_squared_error'] = sklearn.metrics.mean_squared_error(y_true=y_true, y_pred=y_pred)
        scores['mse'] = metrics.mean_squared_error(y_true=y_true, y_pred=y_pred)
        scores['rmse'] = sqrt( metrics.mean_squared_error(y_true=y_true, y_pred=y_pred) )        
        
    scores['y_avg_true'] = np.mean(y_true)
    scores['y_avg_pred'] = np.mean(y_pred)

    return scores

In [14]:
def fnc(prd, mltype='reg'):
    y_pred, y_true = prd['y_pred'], prd['y_true']
    scores = calc_scores(y_true=y_true, y_pred=y_pred, mltype=mltype, metrics=None)
    scores['run'] = str(d).split(os.sep)[-1]
    return scores

In [17]:
# Append scores (dicts)
tr_scores_all = []
vl_scores_all = []
te_scores_all = []

for i, d in enumerate(runs_dirs):
    prd_tr = pd.read_csv(d/'preds_tr.csv')
    prd_vl = pd.read_csv(d/'preds_vl.csv')
    prd_te = pd.read_csv(d/'preds_te.csv')
    # prd['err'] = abs(prd['y_true'] - prd['y_pred'])
    
    mltype = 'reg'
    
    tr_scores = fnc(prd=prd_tr)
    vl_scores = fnc(prd=prd_vl)
    te_scores = fnc(prd=prd_te)
        
    # Append scores (dicts)
    tr_scores_all.append(tr_scores)
    vl_scores_all.append(vl_scores)
    te_scores_all.append(te_scores)
    
    if i%20==0:
        print(f'Processing {d}')

Processing /vol/ml/apartin/projects/DrugResponseViz/notebooks/../out_lgbm/run_s000
Processing /vol/ml/apartin/projects/DrugResponseViz/notebooks/../out_lgbm/run_s020
Processing /vol/ml/apartin/projects/DrugResponseViz/notebooks/../out_lgbm/run_s040
Processing /vol/ml/apartin/projects/DrugResponseViz/notebooks/../out_lgbm/run_s060
Processing /vol/ml/apartin/projects/DrugResponseViz/notebooks/../out_lgbm/run_s080


In [ ]:
def scores_to_df(scores_all):
    scores = pd.DataFrame(scores_all)
    cols = scores.columns.tolist()
    cols.remove('run')
    scores = scores[['run'] + cols]
    return scores

In [21]:
tr_scores_df = scores_to_df(tr_scores_all)
vl_scores_df = scores_to_df(vl_scores_all)
te_scores_df = scores_to_df(te_scores_all)

In [24]:
tr_scores_df.head()

,run,mean_absolute_error,median_absolute_error,mse,r2,rmse,y_avg_pred,y_avg_true
0,run_s000,0.059128,0.041863,0.007048,0.699714,0.083951,0.838018,0.838018
1,run_s001,0.059143,0.041943,0.007019,0.700429,0.083779,0.838169,0.838169
2,run_s002,0.059123,0.041717,0.007049,0.699946,0.083957,0.838163,0.838163
3,run_s003,0.059156,0.041820,0.007049,0.699243,0.083960,0.838202,0.838202
4,run_s004,0.058983,0.041593,0.007017,0.700783,0.083766,0.838191,0.838191
